1. Download voxceleb files from this Drive: https://1drv.ms/u/s!AmYq9m5Um8eJjJwQdIIxEnX4_ELPBA?e=nEcboP 
2. create a function that takes in a wav file and returns a np.array
3. take Sinisa's samples and transform them to array
3. modify the create_databases() function to create dataframes (one df has a speaker_id and mfcc_id), the other has (mfcc_id, mfcc_chanel)

# VoxCeleb Data

In [33]:
import copy

import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from typing import Union
from torch.utils.data import Subset
import time
from tqdm import tqdm
from torchvision import transforms, models
from PIL import Image
from scipy.io.wavfile import read
import wave

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [34]:
import pickle

with open(r"C:\Users\William Hazen\Documents\UofT\Term 1 (F)\MIE 1517\MIE 1517 Projects\audio-speaker-recognition\voxceleb\mfcc_channel_db_64000_16000_13_voxceleb_4s.pkl", 'rb') as f:
    mfcc_channel_db = pickle.load(f)
    
with open(r"C:\Users\William Hazen\Documents\UofT\Term 1 (F)\MIE 1517\MIE 1517 Projects\audio-speaker-recognition\voxceleb\speaker_mfcc_db_64000_16000_13_voxceleb_4s.pkl", 'rb') as f:
    speaker_mfcc_db = pickle.load(f)
speaker_mfcc_db = speaker_mfcc_db.reset_index(drop=True)


# Custom Celeb Data

In [35]:
mypath = r"C:\Users\William Hazen\Documents\UofT\Term 1 (F)\MIE 1517\MIE 1517 Projects\audio-speaker-recognition\Celeb Data"
f = []
for (dirpath, dirnames, filenames) in os.walk(mypath):
    break

In [36]:
print("Folders")
dir_name_list = []
count = 0
for i, dirname in enumerate(dirnames):
    count += 1
    dir_name_list.append(dirname)
    print(i, ' ', dirname)

Folders
0   Alison Pill
1   Amy Poehler
2   Chadwick Boseman
3   Cillian Murphy
4   David Letterman
5   Sinisa


In [37]:

import os

def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

# Run the above function and store its results in a variable.   
full_file_paths = get_filepaths(r"C:\Users\William Hazen\Documents\UofT\Term 1 (F)\MIE 1517\MIE 1517 Projects\audio-speaker-recognition\Celeb Data")

In [38]:
def split_waveform(waveform, window_size):
    """
    Takes an original waveform and reduce to chunks of smaller intervals using a sliding window
    
    waveform : nd.array
    
    window_size : int
        window size in seconds
    
    Returns
    -------
    splitted_waveform : list[nd.array]
        list of waveforms
    
    """
    splitted_waveform = []
    
    for i in range(0, len(waveform), window_size):
        split = waveform[i:i+window_size]
        if len(split) == window_size:
            splitted_waveform.append(split)

    return splitted_waveform

In [39]:
audio_as_np_list = []
sample_rate_list = []
name_list = []
mfccs_list = []
unique_list = []
speaker_ID_list = []
sample_rate = 16000 # 16kHz
window_size = 4 * sample_rate # 1 second has 16000 samples, window_size is 4 seconds
number_spectral_coefficients = 13

def add_custom_samples_to_voxceleb(speaker_mfcc_db, mfcc_channel_db, 
                                window_size, sample_rate, number_spectral_coefficients):
    idx = 300000# mfcc_channel_db.index[-1] + 1

    for filename in full_file_paths:
        #print('processing: ', filename)
        content = filename[113:-4]
        split_name = content.split('\\')[1]
        unique_list.append(split_name[0:4])
        name_list.append(split_name)    
        # if split_name[0:4] == "A_Pi":
        #     speaker_ID = 37
        # elif split_name[0:4] == "A_Po":
        #     speaker_ID = 45 
        # elif split_name[0:4] == "C_Bo":
        #     speaker_ID = 139
        # elif split_name[0:4] == "C_Mu":
        #     speaker_ID = 165
        # elif split_name[0:4] == "D_Le":
        #     speaker_ID = 211
        if split_name[0:4] == "Sini":
            speaker_ID = 8888
            speaker_ID_list.append(speaker_ID)   
            
            waveform, sr = librosa.load(filename, sr=sample_rate)
            waveform_arr = waveform.flatten()
            splitted_waveform = split_waveform(waveform_arr, window_size) # split waveforms into consistent chunks

            for chunk in splitted_waveform: # transform waveforms into MFCC spectrograms
                mfccs = librosa.feature.mfcc(y=chunk.flatten(), n_mfcc=13, sr=sample_rate)

                speaker_mfcc_db.loc[idx, "speaker_id"] = speaker_ID
                speaker_mfcc_db.loc[idx, "mfcc_id"] = idx

                for j in range(number_spectral_coefficients):
                    mfcc_channel_db.loc[idx, f"channel_{j}"] = 1
                    try:
                        mfcc_channel_db.at[idx, f"channel_{j}"] = mfccs[j, :]
                    except ValueError:
                        print(mfccs.shape)
                        # print(mfccs)
                        print(mfccs[j, :].shape)
                        print(idx)
                        print(j)
                        assert False

                idx += 1

    return speaker_mfcc_db, mfcc_channel_db
        # audio_as_np_list.append(y)
        # sample_rate_list.append(sr)

        # # splitted_waveform = split_waveform(y, sr) # split waveforms into consistent chunks

        # # for chunk in splitted_waveform: # transform waveforms into MFCC spectrograms
        # #     mfccs = librosa.feature.mfcc(y=chunk, n_mfcc=13, sr=sr)
        # mfccs = librosa.feature.mfcc(y=y, n_mfcc=13, sr=sr)
        # mfccs_list.append(mfccs)

In [40]:
mfcc_channel_db.loc[98544]


channel_0     [-247.41261, -154.02705, -115.005325, -113.405...
channel_1     [-44.41731, -40.377876, 6.0169764, 65.17988, 7...
channel_2     [45.21265, 54.16138, 50.241623, -5.9958906, -3...
channel_3     [-2.825086, 4.182045, 19.290844, 52.145073, 79...
channel_4     [6.422141, 5.1642165, -0.5214758, -22.322853, ...
channel_5     [11.644709, 8.438548, -19.104036, -27.33027, -...
channel_6     [-15.066701, -8.810129, -15.6354475, -5.974414...
channel_7     [10.318856, 16.105614, 18.073685, -1.7375027, ...
channel_8     [-2.456327, -5.621569, -11.3551445, -14.574454...
channel_9     [-0.54953563, -8.83652, -13.031679, -8.721676,...
channel_10    [-8.665388, -13.082109, -6.7757463, -8.349026,...
channel_11    [-12.409164, -11.899278, -6.3717594, -4.195281...
channel_12    [5.049775, 1.687913, -2.1538005, -8.75869, -7....
Name: 98544, dtype: object

In [41]:
speaker_mfcc_db, mfcc_channel_db = add_custom_samples_to_voxceleb(speaker_mfcc_db, mfcc_channel_db, 
                                window_size, sample_rate, number_spectral_coefficients)

In [42]:
speaker_mfcc_db

,speaker_id,mfcc_id
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
300048,8888,300048
300049,8888,300049
300050,8888,300050
300051,8888,300051


In [44]:
len(speaker_mfcc_db[speaker_mfcc_db["speaker_id"]==8888]) # sinisa's sample ids

53

In [ ]:
speaker_mfcc_db.to_pickle('speaker_mfcc_voxceleb_with_sinisa.pkl')
mfcc_channel_db.to_pickle('mfcc_channel_db_voxceleb_with_sinisa.pkl')


# End

In [ ]:
len(audio_as_np_list), len(sample_rate_list), len(name_list), len(speaker_ID_list), len(mfccs_list)

In [ ]:
celeb_df = pd.DataFrame({
                        "Speaker Name": name_list,
                        "NP Waveforms": audio_as_np_list,
                        "Sample Rate": sample_rate_list,
                        "MFCC's": mfccs_list,
                        "speaker_ID_list": speaker_ID_list,
    
}) 

In [ ]:
celeb_df

- id10039	Alison_Pill (39) -> (37)
- id10046	Amy_Poehler (47) -> (45)
- id10140	Chadwick_Boseman (141) -> (139)
- id10166	Cillian_Murphy	(167) -> (165)
- id10212	David_Letterman (213) -> (211)

In [ ]:
sinisa_df = celeb_df[celeb_df["speaker_ID_list"] == 1251]

In [ ]:
e_sinisa_df = sinisa_df.drop(["NP Waveforms", "Speaker Name", "Sample Rate"], axis=1)
e_sinisa_df["speaker_id"] = e_sinisa_df["speaker_ID_list"]
e_sinisa_df["mfcc"] = e_sinisa_df["MFCC's"]

In [ ]:
list_list = []
for i in range(50,57):
    list_list.append(e_sinisa_df["mfcc"][i])

In [ ]:
len(list_list)
list_list[0]

In [ ]:
e_sinisa_df = e_sinisa_df.drop(["MFCC's", "speaker_ID_list"], axis=1)

In [ ]:
sinina_id = e_sinisa_df.drop(["mfcc"], axis=1)[0:7]
ext = [98544, 98545, 98546, 98547, 98548, 98549, 98550]

sinina_id["mfcc_id"] = [98544, 98545, 98546, 98547, 98548, 98549, 98550]
sinina_id = sinina_id.set_index(sinina_id.mfcc_id)

In [ ]:
temp_speaker_vox_channel = pd.DataFrame(speaker_vox_channel).copy()

In [ ]:
temp_mfcc_vox_channel = pd.DataFrame(mfcc_vox_channel).copy()

In [ ]:
index_val = temp_speaker_vox_channel.index.values.tolist() +  [98544, 98545, 98546, 98547, 98548, 98549, 98550]

In [ ]:
temp_sp_vox_channel = pd.concat([temp_speaker_vox_channel, sinina_id])
temp_sp_vox_channel.set_index(temp_sp_vox_channel.mfcc_id)
temp_sp_vox_channel[-10:]

In [ ]:
temp_mfcc_vox_channel

In [ ]:
sin_mfcc_sp = e_sinisa_df.drop(["speaker_id"], axis=1)[0:7]


In [ ]:
mf_list = sin_mfcc_sp["mfcc"].values
mf_list[0]

In [ ]:
mf_list = sin_mfcc_sp["mfcc"].values

c=0
c1 = 0
channel_list = []
c_list = []
for count in mf_list:
    c_list.append(count)
    for channel in count:
       
       channel_list.append(channel)
       
        

In [ ]:
ch_13_list = []
for batch in range(13):
    ch_13_list = channel_list
    

In [ ]:
len(channel_list)

In [ ]:
len(list_list[0])

In [ ]:
ch_list = ["channel_0", "channel_1", "channel_2","channel_3","channel_4","channel_5","channel_6","channel_7","channel_8", "channel_9", "channel_10", "channel_11", "channel_12"]
ext = [98544, 98545, 98546, 98547, 98548, 98549, 98550]

In [ ]:
df_temp = pd.DataFrame(columns=ch_list, index=ext)
df_temp.loc[98544] = list_list[0].tolist()
df_temp.loc[98545] = list_list[1].tolist()
df_temp.loc[98546] = list_list[2].tolist()
df_temp.loc[98547] = list_list[3].tolist()
df_temp.loc[98548] = list_list[4].tolist()
df_temp.loc[98549] = list_list[5].tolist()
df_temp.loc[98550] = list_list[6].tolist()
df_temp.rename_axis('mfcc_id')

In [ ]:
# sin_mfcc_sp["mfcc_id"] = [98544, 98545, 98546, 98547, 98548, 98549, 98550]
# sin_mfcc_sp = sin_mfcc_sp.set_index(sin_mfcc_sp.mfcc_id)
# sin_mfcc_sp.drop(["mfcc_id", "mfcc"],axis=1)
# sin_mfcc_sp["channel_0"] = list_list


In [ ]:
temp_mfcc_vox_channel = pd.concat([temp_mfcc_vox_channel, df_temp])

In [ ]:
def create_mfcc_databases(dataset, window_size, sample_rate, speaker_dict, number_spectral_coefficients):
    """
    Creates two databases from an audio dataset e.g. librispeech
    
    Inputs
    -----
    
    base_dataset : 
        A pytorch audio dataset like librispeech
    
    window_size : int
        window size to be used to consistently split waveforms

    sample_rate : int
        sample rate in kHz    
    
    speaker_dict : dict
        dictionary of {'speaker_id':encoded_id} pairs          

    number_spectral_coefficients : int
        number of spectral coefficients to keep for the MFCC calculation
        
    Returns
    -------
    speaker_mfcc_db : pd.DataFrame 
        df with shape (index_id, speaker_id, mfcc_id)
    
    mfcc_channel_db : pd.DataFrame
        df with shape (mfcc_id, channel_id)
    
    """
    try:
        speaker_mfcc_db = pd.read_pickle(f'speaker_mfcc_db_{window_size}_{sample_rate}_{number_spectral_coefficients}.pkl')
        mfcc_channel_db = pd.read_pickle(f'mfcc_channel_db_{window_size}_{sample_rate}_{number_spectral_coefficients}.pkl')
    
    except FileNotFoundError:
        speaker_mfcc_db = pd.DataFrame(columns=["speaker_id", "mfcc_id"])
        mfcc_channel_db = pd.DataFrame(columns=["mfcc_id"]+[f"channel_{i}" for i in range(number_spectral_coefficients)]).set_index('mfcc_id')

        idx = 0

        print('Creating a database. Hold tight...')
        for i in tqdm(range(dataset.__len__())):
            (waveform, sample_rate, transcript, speaker_id, chapter_id, utterance_id) = dataset.__getitem__(i)
            waveform_arr = waveform.numpy().flatten()
            splitted_waveform = split_waveform(waveform_arr, window_size) # split waveforms into consistent chunks

            for chunk in splitted_waveform: # transform waveforms into MFCC spectrograms
                mfccs = librosa.feature.mfcc(y=chunk.flatten(), n_mfcc=13, sr=sample_rate)

                speaker_mfcc_db.loc[idx, "speaker_id"] = speaker_dict[speaker_id]
                speaker_mfcc_db.loc[idx, "mfcc_id"] = idx

                for j in range(number_spectral_coefficients):
                    mfcc_channel_db.loc[idx, f"channel_{j}"] = 1
                    mfcc_channel_db.at[idx, f"channel_{j}"] = mfccs[j, :]
                
                idx += 1

        # speaker_mfcc_db.to_pickle(f'speaker_mfcc_db_{window_size}_{sample_rate}_{number_spectral_coefficients}.pkl')
        # mfcc_channel_db.to_pickle(f'mfcc_channel_db_{window_size}_{sample_rate}_{number_spectral_coefficients}.pkl')

    return speaker_mfcc_db, mfcc_channel_db

In [ ]:
create_mfcc_databases()

In [ ]:
first = [x for x in ((temp_mfcc_vox_channel["channel_0"][-6:-5]).values)]
for x in first:
    print(len(x))

In [ ]:
# with open('E_Pickle.pkl', 'wb') as f:
#     pickle.dump(temp_sp_vox_channel, f)

# with open('S_Pickle.pkl', 'wb') as f:
#     pickle.dump(S, f)